# 💾 Sesión 04: Serialización de Modelos - Del Notebook a Producción

## 📑 Índice
1.  [¿Por Qué Serializar?](#1.-¿Por-Qué-Serializar?) 🤔
2.  [Nivel 1: Pickle (Lo Básico)](#2.-Nivel-1:-Pickle) 🥒
3.  [Nivel 2: Joblib (El Estándar ML)](#3.-Nivel-2:-Joblib) 📦
4.  [Nivel 3: ONNX (Interoperabilidad)](#4.-Nivel-3:-ONNX) 🌐
5.  [Nivel 4: PMML (Legacy Enterprise)](#5.-Nivel-4:-PMML) 🏢
6.  [Nivel 5: Formatos Nativos](#6.-Nivel-5:-Formatos-Nativos) 🎯
7.  [Comparativa Final](#7.-Comparativa-Final) 📊
8.  [Micro-Desafío](#8.-🧠-Micro-Desafío) 🧠

## 💼 Caso de Negocio: El Modelo que Nadie Pudo Desplegar
**Contexto:**
Tu modelo de Credit Scoring tiene un AUC de 0.98. ¡El Gerente está feliz!
Pero llega el equipo de Infraestructura y pregunta: *"¿En qué formato está? Nuestra API está en Java, el frontend en JavaScript, y el sistema legacy en C++."*

**El Problema:**
El modelo está en un `.pkl` de Python. Solo funciona si tienes **exactamente** la misma versión de Python, Scikit-Learn y LightGBM. En otro lenguaje... simplemente **no carga**.

**Tu Misión:**
Aprender **5 formas de serializar** modelos, desde la más simple hasta la más portable, para que tu modelo pueda desplegarse **en cualquier entorno**.

## 🎯 Objetivos de Aprendizaje
| Nivel | Objetivo |
|-------|----------|
| 🟢 Básico | Entender Pickle y Joblib para entornos Python |
| 🟡 Intermedio | Exportar a ONNX para APIs multi-lenguaje |
| 🔴 Avanzado | Usar formatos nativos (LightGBM txt) para máxima portabilidad |

In [ ]:
# Librerías
from recursos.utils import load_data
import sys
import os
import pickle
import json
import time

import pandas as pd
import numpy as np
import joblib
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

# Cargar utilidades del curso
sys.path.append(os.path.abspath('../../'))

# Crear carpeta para artefactos
os.makedirs('../app/models', exist_ok=True)
print("✅ Librerías cargadas")


✅ Librerías cargadas


## 1. ¿Por Qué Serializar? 🤔

**Serialización** = Convertir un objeto en memoria (tu modelo) a un formato que puede guardarse en disco y cargarse después.

### El Ciclo de Vida de un Modelo
```
[Entrenamiento] → [Serialización] → [Almacenamiento] → [Carga] → [Inferencia]
     Python           ???              Disco/S3         ???        API/App
```

### ¿Por Qué No Basta con Re-entrenar?
| Problema | Consecuencia |
|----------|--------------|
| Entrenar toma tiempo | Latencia inaceptable en APIs |
| Los datos cambian | Resultados no reproducibles |
| Dependencia del entorno | "En mi máquina funciona..." |

---
## Preparación: Entrenar el Modelo Base

Primero entrenamos un modelo LightGBM que usaremos para probar todos los formatos.

In [ ]:
# Cargar y preparar datos
df = load_data('credit_scoring.csv')

TARGET_COL = 'target_y'
COLS_TO_DROP = [TARGET_COL, 'malo_sf_inicio', 'periodo', 'Unnamed: 0']
COLS_SELECT = ['SD_MAX_DIAS_MORA_SSFF_06M',
               'MAX_PORC_DEUDA_SOBREGIRO_CUENTA_CORRIENTE_ENTFIN_12M',
               'MAX_CNT_ENTIDADES_SSFF_06M',
               'NumeroTrabajadores',
               'ANTIGUEDAD_RCC_01M']

X = df[COLS_SELECT]
y = df[TARGET_COL]

# Solo numéricas para simplificar
X = X.select_dtypes(include=['int64', 'float64'])
X = X.fillna(X.median())

# Split para validación
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# Guardar nombres de features (metadata crítico)
FEATURE_NAMES = X.columns.tolist()
print(f"📊 Dataset: {X.shape[0]} filas, {len(FEATURE_NAMES)} features")

# Entrenar modelo
print("\n🚀 Entrenando LightGBM...")
model = lgb.LGBMClassifier(n_estimators=100, random_state=42, verbose=-1)
model.fit(X_train, y_train)

# Validar
auc = roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])
print(f"✅ AUC en Test: {auc:.4f}")


📊 Dataset: 7180 filas, 5 features

🚀 Entrenando LightGBM...
✅ AUC en Test: 0.9425


---
## 2. Nivel 1: Pickle 🥒 (Lo Básico)

**Pickle** es el serializador nativo de Python. Convierte cualquier objeto Python a bytes.

### ✅ Ventajas
- Viene incluido en Python (no requiere instalar nada)
- Funciona con casi cualquier objeto Python

### ❌ Desventajas
- **Solo Python** (no puedes cargar en Java, JS, etc.)
- **Frágil:** Si cambias la versión de scikit-learn, puede fallar
- **Inseguro:** Puede ejecutar código malicioso al cargar

In [ ]:
# ========================================
# MÉTODO 1: PICKLE (Básico)
# ========================================
pickle_path = '../app/models/model_pickle.pkl'

# Guardar
start = time.time()
with open(pickle_path, 'wb') as f:
    pickle.dump(model, f)
pickle_save_time = time.time() - start

# Cargar
start = time.time()
with open(pickle_path, 'rb') as f:
    model_pickle = pickle.load(f)
pickle_load_time = time.time() - start

# Verificar
pred_pickle = model_pickle.predict_proba(X_test)[:, 1]
auc_pickle = roc_auc_score(y_test, pred_pickle)

pickle_size = os.path.getsize(pickle_path) / 1024  # KB

print(f"📁 Archivo: {pickle_path}")
print(f"📦 Tamaño: {pickle_size:.2f} KB")
print(f"⏱️ Tiempo guardar: {pickle_save_time*1000:.2f} ms")
print(f"⏱️ Tiempo cargar: {pickle_load_time*1000:.2f} ms")
print(f"✅ AUC verificado: {auc_pickle:.4f}")


📁 Archivo: ../app/models/model_pickle.pkl
📦 Tamaño: 335.48 KB
⏱️ Tiempo guardar: 18.83 ms
⏱️ Tiempo cargar: 34.94 ms
✅ AUC verificado: 0.9425


### ⚠️ Real-World Warning: Pickle y Seguridad
**NUNCA** cargues un archivo `.pkl` de una fuente no confiable.
Pickle puede ejecutar código arbitrario al deserializar. Un atacante podría crear un `.pkl` malicioso que borre tu disco o robe credenciales.

```python
# ❌ PELIGRO: No hagas esto con archivos de internet
model = pickle.load(open('modelo_de_internet.pkl', 'rb'))  # Podría ejecutar malware
```

---
## 3. Nivel 2: Joblib 📦 (El Estándar ML)

**Joblib** es una versión optimizada de Pickle para objetos con arrays grandes (como modelos de ML).

### ✅ Ventajas
- **Más rápido** que Pickle para modelos grandes
- **Compresión** integrada (reduce tamaño en disco)
- Estándar en la comunidad ML

### ❌ Desventajas
- Mismos problemas de portabilidad que Pickle
- Sigue siendo solo Python

In [ ]:
# ========================================
# MÉTODO 2: JOBLIB (Estándar ML)
# ========================================
joblib_path = '../app/models/model_joblib.joblib'

# Guardar CON metadatos (buena práctica)
artifact = {
    'model': model,
    'feature_names': FEATURE_NAMES,
    'target_col': TARGET_COL,
    'auc_test': auc,
    'training_date': pd.Timestamp.now().isoformat(),
    'python_version': sys.version,
    'lightgbm_version': lgb.__version__
}

start = time.time()
joblib.dump(artifact, joblib_path, compress=3)  # compress=3 es buen balance
joblib_save_time = time.time() - start

# Cargar
start = time.time()
loaded_artifact = joblib.load(joblib_path)
joblib_load_time = time.time() - start

model_joblib = loaded_artifact['model']

# Verificar
pred_joblib = model_joblib.predict_proba(X_test)[:, 1]
auc_joblib = roc_auc_score(y_test, pred_joblib)

joblib_size = os.path.getsize(joblib_path) / 1024

print(f"📁 Archivo: {joblib_path}")
print(f"📦 Tamaño: {joblib_size:.2f} KB (comprimido)")
print(f"⏱️ Tiempo guardar: {joblib_save_time*1000:.2f} ms")
print(f"⏱️ Tiempo cargar: {joblib_load_time*1000:.2f} ms")
print(f"✅ AUC verificado: {auc_joblib:.4f}")
print(f"\n📋 Metadatos guardados: {list(loaded_artifact.keys())}")


📁 Archivo: ../app/models/model_joblib.joblib
📦 Tamaño: 141.72 KB (comprimido)
⏱️ Tiempo guardar: 22.80 ms
⏱️ Tiempo cargar: 31.77 ms
✅ AUC verificado: 0.9425

📋 Metadatos guardados: ['model', 'feature_names', 'target_col', 'auc_test', 'training_date', 'python_version', 'lightgbm_version']


> **💡 Pro-Tip: Siempre guarda metadatos**
> Incluir `feature_names`, `training_date`, y versiones de librerías te salvará cuando en 6 meses alguien pregunte "¿Con qué datos se entrenó esto?"

---
## 4. Nivel 3: ONNX 🌐 (Interoperabilidad)

**ONNX (Open Neural Network Exchange)** es un formato abierto para representar modelos de ML.

### ✅ Ventajas
- **Multi-lenguaje:** Carga en Python, C++, Java, JavaScript, C#
- **Optimizado:** Runtime ONNX es muy rápido
- **Estándar de la industria:** Microsoft, Facebook, Amazon lo usan

### ❌ Desventajas
- Requiere conversión (no todos los modelos son compatibles)
- Necesita instalar `onnxmltools` y `onnxruntime`

### 🌍 Lenguajes Soportados
| Lenguaje | Runtime |
|----------|---------|
| Python | `onnxruntime` |
| C++ | ONNX Runtime C++ |
| Java | `onnxruntime-java` |
| JavaScript | `onnxruntime-web` |
| C# | `Microsoft.ML.OnnxRuntime` |

In [ ]:
# ========================================
# MÉTODO 3: ONNX (Multi-lenguaje)
# ========================================
try:
    import onnxmltools
    from onnxmltools.convert import convert_lightgbm
    from onnxconverter_common import FloatTensorType
    import onnxruntime as ort
    ONNX_AVAILABLE = True
except ImportError:
    ONNX_AVAILABLE = False
    print("⚠️ ONNX no instalado. Ejecuta: pip install onnxmltools onnxruntime")

if ONNX_AVAILABLE:
    onnx_path = '../app/models/model_onnx.onnx'

    # Definir el tipo de entrada (n_features como float)
    initial_type = [('input', FloatTensorType([None, len(FEATURE_NAMES)]))]

    # Convertir modelo
    start = time.time()
    onnx_model = convert_lightgbm(
        model.booster_,  # LightGBM interno
        initial_types=initial_type,
        target_opset=12
    )

    # Guardar
    with open(onnx_path, 'wb') as f:
        f.write(onnx_model.SerializeToString())
    onnx_save_time = time.time() - start

    # Cargar con ONNX Runtime
    start = time.time()
    ort_session = ort.InferenceSession(onnx_path)
    onnx_load_time = time.time() - start

    # Inferencia
    input_name = ort_session.get_inputs()[0].name
    X_test_np = X_test.values.astype(np.float32)

    start = time.time()
    onnx_output = ort_session.run(None, {input_name: X_test_np})
    onnx_inference_time = time.time() - start

    # ONNX devuelve [labels, probabilities]
    pred_onnx = onnx_output[1][:, 1]  # Probabilidad clase 1
    auc_onnx = roc_auc_score(y_test, pred_onnx)

    onnx_size = os.path.getsize(onnx_path) / 1024

    print(f"📁 Archivo: {onnx_path}")
    print(f"📦 Tamaño: {onnx_size:.2f} KB")
    print(f"⏱️ Tiempo conversión+guardado: {onnx_save_time*1000:.2f} ms")
    print(f"⏱️ Tiempo cargar: {onnx_load_time*1000:.2f} ms")
    print(
        f"⏱️ Tiempo inferencia ({len(X_test)} muestras): {onnx_inference_time*1000:.2f} ms")
    print(f"✅ AUC verificado: {auc_onnx:.4f}")


⚠️ ONNX no instalado. Ejecuta: pip install onnxmltools onnxruntime


### 📝 Ejemplo: Cargar ONNX en JavaScript (Node.js)
```javascript
// npm install onnxruntime-node
const ort = require('onnxruntime-node');

async function predict(features) {
    const session = await ort.InferenceSession.create('model_onnx.onnx');
    const tensor = new ort.Tensor('float32', features, [1, 21]);
    const results = await session.run({ input: tensor });
    return results.probabilities.data[1];  // P(default)
}
```

### 📝 Ejemplo: Cargar ONNX en C++
```cpp
#include <onnxruntime_cxx_api.h>

Ort::Env env(ORT_LOGGING_LEVEL_WARNING, "CreditModel");
Ort::Session session(env, "model_onnx.onnx", Ort::SessionOptions{});
// ... preparar input tensor y ejecutar
```

---
## 5. Nivel 4: PMML 🏢 (Legacy Enterprise)

**PMML (Predictive Model Markup Language)** es un formato XML antiguo pero aún usado en sistemas bancarios legacy.

### ✅ Ventajas
- Ampliamente soportado en sistemas empresariales (SAS, SPSS, Java)
- Formato de texto (legible y auditable)

### ❌ Desventajas
- **Lento** (XML es verbose)
- No soporta modelos muy complejos (deep learning)
- Menos preciso que formatos binarios

In [ ]:
# ========================================
# MÉTODO 4: PMML (Enterprise/Legacy)
# ========================================
try:
    from sklearn2pmml import sklearn2pmml
    from sklearn2pmml.pipeline import PMMLPipeline
    PMML_AVAILABLE = True
except ImportError:
    PMML_AVAILABLE = False
    print("⚠️ sklearn2pmml no instalado. Ejecuta: pip install sklearn2pmml")
    print("   También requiere Java instalado.")

# Nota: PMML requiere Java y configuración especial.
# Mostramos el código de referencia:
print("""
📋 Código de referencia para PMML (requiere Java):

from sklearn2pmml import sklearn2pmml
from sklearn2pmml.pipeline import PMMLPipeline

# Crear pipeline compatible con PMML
pmml_pipeline = PMMLPipeline([
    ("classifier", lgb.LGBMClassifier(n_estimators=100))
])
pmml_pipeline.fit(X_train, y_train)

# Exportar
sklearn2pmml(pmml_pipeline, "model.pmml", with_repr=True)
""")


⚠️ sklearn2pmml no instalado. Ejecuta: pip install sklearn2pmml
   También requiere Java instalado.

📋 Código de referencia para PMML (requiere Java):

from sklearn2pmml import sklearn2pmml
from sklearn2pmml.pipeline import PMMLPipeline

# Crear pipeline compatible con PMML
pmml_pipeline = PMMLPipeline([
    ("classifier", lgb.LGBMClassifier(n_estimators=100))
])
pmml_pipeline.fit(X_train, y_train)

# Exportar
sklearn2pmml(pmml_pipeline, "model.pmml", with_repr=True)



---
## 6. Nivel 5: Formatos Nativos 🎯 (Máxima Portabilidad)

Cada librería tiene su **formato nativo** optimizado. Para LightGBM, es un archivo de texto que contiene la estructura de todos los árboles.

### ✅ Ventajas
- **Máxima portabilidad:** Carga en C++, Java, Go, Rust (cualquier binding de LightGBM)
- **Sin dependencias de Python:** Perfecto para embebidos o microservicios
- **Texto plano:** Auditable y versionable en Git

### ❌ Desventajas
- Específico de cada librería (LightGBM ≠ XGBoost ≠ CatBoost)
- No incluye preprocesamiento (solo el modelo)

In [ ]:
# ========================================
# MÉTODO 5A: LightGBM Nativo (Texto)
# ========================================
lgb_txt_path = '../app/models/model_lgb.txt'

# Guardar en formato texto nativo
start = time.time()
model.booster_.save_model(lgb_txt_path)
lgb_txt_save_time = time.time() - start

# Cargar
start = time.time()
model_lgb_txt = lgb.Booster(model_file=lgb_txt_path)
lgb_txt_load_time = time.time() - start

# Verificar
pred_lgb_txt = model_lgb_txt.predict(X_test)
auc_lgb_txt = roc_auc_score(y_test, pred_lgb_txt)

lgb_txt_size = os.path.getsize(lgb_txt_path) / 1024

print(f"📁 Archivo: {lgb_txt_path}")
print(f"📦 Tamaño: {lgb_txt_size:.2f} KB")
print(f"⏱️ Tiempo guardar: {lgb_txt_save_time*1000:.2f} ms")
print(f"⏱️ Tiempo cargar: {lgb_txt_load_time*1000:.2f} ms")
print(f"✅ AUC verificado: {auc_lgb_txt:.4f}")


📁 Archivo: ../app/models/model_lgb.txt
📦 Tamaño: 333.72 KB
⏱️ Tiempo guardar: 27.37 ms
⏱️ Tiempo cargar: 79.38 ms
✅ AUC verificado: 0.9425


In [ ]:
# Veamos cómo se ve el archivo de texto
print("📄 Primeras 30 líneas del modelo LightGBM (texto):\n")
with open(lgb_txt_path, 'r') as f:
    for i, line in enumerate(f):
        if i < 30:
            print(line.rstrip())
        else:
            print("... (continúa)")
            break


📄 Primeras 30 líneas del modelo LightGBM (texto):

tree
version=v4
num_class=1
num_tree_per_iteration=1
label_index=0
max_feature_idx=4
objective=binary sigmoid:1
feature_names=SD_MAX_DIAS_MORA_SSFF_06M MAX_PORC_DEUDA_SOBREGIRO_CUENTA_CORRIENTE_ENTFIN_12M MAX_CNT_ENTIDADES_SSFF_06M NumeroTrabajadores ANTIGUEDAD_RCC_01M
feature_infos=[0:192.61092042422399] [1.14e-08:1] [1:10] [0:1433] [0:57]
tree_sizes=3324 3335 3349 3348 3368 3359 3384 3360 3364 3356 3357 3363 3365 3372 3370 3383 3362 3371 3394 3366 3373 3388 3391 3366 3393 3381 3385 3387 3387 3394 3379 3376 3368 3367 3393 3380 3361 3380 3384 3377 3384 3391 3384 3388 3365 3366 3396 3395 3362 3401 3376 3357 3401 3364 3397 3392 3376 3397 3385 3426 3400 3366 3388 3399 3375 3387 3431 3385 3393 3363 3368 3362 3389 3389 3423 3373 3391 3396 3417 3377 3395 3348 3371 3381 3403 3379 3385 3428 3358 3374 3367 3387 3395 3389 3378 3390 3435 3378 3396 3405

Tree=0
num_leaves=31
num_cat=0
split_feature=0 2 0 1 0 3 2 1 1 2 1 3 0 0 4 3 0 1 0 4 2 0 1 3 1

In [ ]:
# ========================================
# MÉTODO 5B: JSON + Metadatos (Para APIs)
# ========================================
json_path = '../app/models/model_metadata.json'

# Guardar metadatos en JSON (legible por cualquier lenguaje)
metadata = {
    'model_type': 'LightGBMClassifier',
    'model_file': 'model_lgb.txt',
    'feature_names': FEATURE_NAMES,
    'n_features': len(FEATURE_NAMES),
    'target_col': TARGET_COL,
    'metrics': {
        'auc_test': round(auc, 4),
        'n_train_samples': len(X_train),
        'n_test_samples': len(X_test)
    },
    'training_info': {
        'date': pd.Timestamp.now().isoformat(),
        'lightgbm_version': lgb.__version__,
        'n_estimators': model.n_estimators
    },
    'feature_importance': dict(zip(
        FEATURE_NAMES,
        model.feature_importances_.tolist()
    ))
}

with open(json_path, 'w') as f:
    json.dump(metadata, f, indent=2)

print(f"📁 Metadatos guardados: {json_path}")
print(f"\n📋 Contenido:")
print(json.dumps(metadata, indent=2)[:1000] + "...")


📁 Metadatos guardados: ../app/models/model_metadata.json

📋 Contenido:
{
  "model_type": "LightGBMClassifier",
  "model_file": "model_lgb.txt",
  "feature_names": [
    "SD_MAX_DIAS_MORA_SSFF_06M",
    "MAX_PORC_DEUDA_SOBREGIRO_CUENTA_CORRIENTE_ENTFIN_12M",
    "MAX_CNT_ENTIDADES_SSFF_06M",
    "NumeroTrabajadores",
    "ANTIGUEDAD_RCC_01M"
  ],
  "n_features": 5,
  "target_col": "target_y",
  "metrics": {
    "auc_test": 0.9425,
    "n_train_samples": 5744,
    "n_test_samples": 1436
  },
  "training_info": {
    "date": "2025-12-06T02:54:35.175460",
    "lightgbm_version": "4.6.0",
    "n_estimators": 100
  },
  "feature_importance": {
    "SD_MAX_DIAS_MORA_SSFF_06M": 676,
    "MAX_PORC_DEUDA_SOBREGIRO_CUENTA_CORRIENTE_ENTFIN_12M": 715,
    "MAX_CNT_ENTIDADES_SSFF_06M": 263,
    "NumeroTrabajadores": 839,
    "ANTIGUEDAD_RCC_01M": 507
  }
}...


### 📝 Ejemplo: Cargar en C++ (LightGBM Nativo)
```cpp
#include <LightGBM/c_api.h>

BoosterHandle booster;
LGBM_BoosterCreateFromModelfile("model_lgb.txt", &num_iterations, &booster);

// Predecir
double prediction;
LGBM_BoosterPredictForMat(booster, data, ...);
```

### 📝 Ejemplo: Cargar en Java
```java
import ml.dmlc.lightgbm4j.LightGBM;

Booster booster = Booster.loadModel("model_lgb.txt");
double[] predictions = booster.predict(features);
```

---
## 7. Comparativa Final 📊

Creemos una tabla resumen con todos los métodos:

In [ ]:
# ========================================
# COMPARATIVA FINAL
# ========================================

comparison_data = {
    'Formato': ['Pickle', 'Joblib', 'ONNX', 'LightGBM Nativo'],
    'Tamaño (KB)': [
        round(pickle_size, 2),
        round(joblib_size, 2),
        round(onnx_size, 2) if ONNX_AVAILABLE else 'N/A',
        round(lgb_txt_size, 2)
    ],
    'T. Guardar (ms)': [
        round(pickle_save_time*1000, 2),
        round(joblib_save_time*1000, 2),
        round(onnx_save_time*1000, 2) if ONNX_AVAILABLE else 'N/A',
        round(lgb_txt_save_time*1000, 2)
    ],
    'T. Cargar (ms)': [
        round(pickle_load_time*1000, 2),
        round(joblib_load_time*1000, 2),
        round(onnx_load_time*1000, 2) if ONNX_AVAILABLE else 'N/A',
        round(lgb_txt_load_time*1000, 2)
    ],
    'Multi-Lenguaje': ['❌ Solo Python', '❌ Solo Python', '✅ Sí', '✅ Sí'],
    'Incluye Metadata': ['❌', '✅ (manual)', '❌', '❌'],
    'Uso Recomendado': [
        'Prototipos rápidos',
        'Producción Python',
        'APIs multi-lenguaje',
        'Microservicios C++/Java'
    ]
}

comparison_df = pd.DataFrame(comparison_data)
print("📊 COMPARATIVA DE FORMATOS DE SERIALIZACIÓN")
print("=" * 80)
display(comparison_df)


📊 COMPARATIVA DE FORMATOS DE SERIALIZACIÓN


,Formato,Tamaño (KB),T. Guardar (ms),T. Cargar (ms),Multi-Lenguaje,Incluye Metadata,Uso Recomendado
0,Pickle,335.48,18.83,34.94,❌ Solo Python,❌,Prototipos rápidos
1,Joblib,141.72,22.8,31.77,❌ Solo Python,✅ (manual),Producción Python
2,ONNX,N/A,N/A,N/A,✅ Sí,❌,APIs multi-lenguaje
3,LightGBM Nativo,333.72,27.37,79.38,✅ Sí,❌,Microservicios C++/Java


### 🎯 Árbol de Decisión: ¿Qué Formato Usar?

```
¿Tu API es Python?
    │
    ├── SÍ → ¿Modelo grande (>100MB)?
    │           │
    │           ├── SÍ → Joblib (comprimido)
    │           └── NO → Joblib (estándar)
    │
    └── NO → ¿Necesitas máxima velocidad?
                │
                ├── SÍ → Formato Nativo (LightGBM txt, XGBoost json)
                └── NO → ONNX (más flexible)
```

### ⚠️ Real-World Warning: Versionamiento de Modelos
En producción, **SIEMPRE** incluye metadatos con cada modelo:
- Versión de librerías usadas
- Fecha de entrenamiento
- Hash de los datos de entrenamiento
- Métricas de validación

Sin esto, en 6 meses no sabrás qué modelo es cuál.

---
## 8. 🧠 Micro-Desafío: Exportar para tu Stack

Elige el formato correcto para cada escenario:

1. **Escenario A:** Tu equipo de Backend usa FastAPI (Python) y necesita servir predicciones con latencia < 50ms.

2. **Escenario B:** El equipo móvil quiere ejecutar el modelo directamente en la app iOS (Swift).

3. **Escenario C:** El banco tiene un sistema legacy en COBOL que necesita consumir las predicciones.

> **💡 Pista para B:** Busca "Core ML" de Apple. ONNX puede convertirse a Core ML.

---
## 🏆 Resumen de Logros

¡Felicidades! Ahora sabes:
1. **Pickle/Joblib:** Para entornos Python puros
2. **ONNX:** El formato universal para multi-lenguaje
3. **Formatos Nativos:** Para máximo rendimiento en C++/Java
4. **La importancia de metadatos:** Sin ellos, el modelo es una caja negra

### 📁 Archivos Generados
```
app/models/
├── model_pickle.pkl      # Python básico
├── model_joblib.joblib   # Python + metadatos
├── model_onnx.onnx       # Multi-lenguaje
├── model_lgb.txt         # LightGBM nativo
└── model_metadata.json   # Documentación
```

👉 **Siguiente Paso:** Ahora que tienes el modelo serializado, vamos a crear una **aplicación web con Streamlit** que lo consuma en tiempo real.